# **Final Implementation** 

In [ ]:
# ============================================================
# 1. Import Libraries
# ============================================================
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2
import torch.nn.functional as F
import pandas as pd
from datetime import datetime

# ============================================================
# 2. Define Dataset Paths (update this to your path)
# ============================================================
BASE_DIR = "/kaggle/input/brain-tumor-mri-dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "Training")
TEST_DIR = os.path.join(BASE_DIR, "Testing")

# ============================================================
# 3. Define Classes
# ============================================================
CLASSES = ['glioma', 'meningioma', 'notumor', 'pituitary']

# ============================================================
# 4. Transformations
# ============================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ============================================================
# 5. Custom Dataset
# ============================================================
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        for idx, cls in enumerate(CLASSES):
            cls_dir = os.path.join(root_dir, cls)
            for img_name in os.listdir(cls_dir):
                img_path = os.path.join(cls_dir, img_name)
                self.images.append(img_path)
                self.labels.append(idx)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# ============================================================
# 6. Load Datasets
# ============================================================
train_dataset = BrainTumorDataset(TRAIN_DIR, transform=train_transform)
test_dataset = BrainTumorDataset(TEST_DIR, transform=test_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# ============================================================
# 7. Define Model (EfficientNet-B0)
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(pretrained=True)
# Modify classifier
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(CLASSES))
model = model.to(device)

# ============================================================
# 8. Training Setup
# ============================================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# ============================================================
# 9. Train Model and Track Metrics
# ============================================================
epochs = 5
train_losses = []
train_accs = []
test_losses = []
test_accs = []

for epoch in range(epochs):
    # Training
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_train_loss = running_loss / len(train_loader)
    epoch_train_acc = 100 * correct / total
    train_losses.append(epoch_train_loss)
    train_accs.append(epoch_train_acc)
    
    # Test (using test set as validation proxy)
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_test_loss = test_loss / len(test_loader)
    epoch_test_acc = 100 * correct / total
    test_losses.append(epoch_test_loss)
    test_accs.append(epoch_test_acc)
    
    print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, "
          f"Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_acc:.2f}%")

# Save model for versioning
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_path = f"efficientnet_b0_brain_tumor_{timestamp}.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved as {model_path} for versioning.")

# ============================================================
# 10. Evaluate Model with Confidence and Logging
# ============================================================
model.eval()
y_true, y_pred = [], []
all_images = []
all_labels = []
all_confidences = []
log_data = []  # For logging

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)
        confs, preds = torch.max(probs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        all_images.extend(images.cpu())
        all_labels.extend(labels.cpu().numpy())
        all_confidences.extend(confs.cpu().numpy())
        
        # Log per batch
        for i in range(len(labels)):
            log_entry = {
                'true_label': CLASSES[labels[i].item()],
                'predicted_label': CLASSES[preds[i].item()],
                'confidence': confs[i].item()
            }
            log_data.append(log_entry)

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=CLASSES))

# Save logs to CSV
log_df = pd.DataFrame(log_data)
log_df.to_csv(f"prediction_logs_{timestamp}.csv", index=False)
print(f"Prediction logs saved to prediction_logs_{timestamp}.csv")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=CLASSES, yticklabels=CLASSES)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# ============================================================
# 11. Plot Loss and Accuracy Curves
# ============================================================
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curves')
plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc')
plt.plot(test_accs, label='Test Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curves')
plt.show()

# ============================================================
# 12. Bias-Aware Grad-CAM Explainability with Adaptive Sampling
# ============================================================

# Define Bias-Aware Grad-CAM class (with adaptive sampling via perturbations)
class BiasAwareGradCAM:
    def __init__(self, model, target_layer, num_samples=1, noise_std=0.01):
        self.model = model
        self.target_layer = target_layer
        self.num_samples = num_samples  # Number of perturbations for averaging
        self.noise_std = noise_std  # Standard deviation for Gaussian noise
        self.gradients = None
        self.activations = None
        
        # Register hooks
        self.target_layer.register_forward_hook(self.save_activations)
        self.target_layer.register_full_backward_hook(self.save_gradients)
    
    def save_activations(self, module, input, output):
        self.activations = output
    
    def save_gradients(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]
    
    def generate(self, input_image, class_idx=None):
        grad_cam_maps = []
        for _ in range(self.num_samples):
            # Add Gaussian noise for perturbation (bias-aware sampling)
            perturbed_image = input_image + torch.randn_like(input_image) * self.noise_std
            perturbed_image = torch.clamp(perturbed_image, 0, 1)  # Clamp to valid range
            
            self.model.eval()
            # Forward pass
            output = self.model(perturbed_image)
            if class_idx is None:
                class_idx = torch.argmax(output, dim=1).item()
            
            # Zero gradients
            self.model.zero_grad()
            # Backward pass for the specific class
            output[:, class_idx].backward()
            
            # Compute Grad-CAM
            weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
            grad_cam = F.relu(torch.sum(weights * self.activations, dim=1)).squeeze()
            grad_cam = grad_cam.detach().cpu().numpy()  # Detach before converting to numpy
            
            # Normalize Grad-CAM
            grad_cam = np.maximum(grad_cam, 0)
            grad_cam = grad_cam / (grad_cam.max() + 1e-8)
            grad_cam_maps.append(grad_cam)
        
        # Average over samples for bias-aware explanation
        avg_grad_cam = np.mean(grad_cam_maps, axis=0)
        return avg_grad_cam, class_idx, output

# Initialize Bias-Aware Grad-CAM (with 3 samples for adaptive feature sampling)
target_layer = model.features[-1]  # Last conv layer in EfficientNet-B0
grad_cam = BiasAwareGradCAM(model, target_layer, num_samples=3, noise_std=0.01)

# Find one example index per class from the test set
class_examples = {}
for i, label in enumerate(all_labels):
    if label not in class_examples:
        class_examples[label] = i
    if len(class_examples) == len(CLASSES):
        break

# Generate and plot Bias-Aware Grad-CAM for one example per class, with confidence
for class_idx, example_idx in class_examples.items():
    input_image = all_images[example_idx].unsqueeze(0).to(device)  # Add batch dimension
    true_label = all_labels[example_idx]
    
    grad_cam_map, predicted_class, output = grad_cam.generate(input_image)
    confidence = F.softmax(output, dim=1)[0, predicted_class].item()
    
    # Resize Grad-CAM map to match input image size
    grad_cam_map = cv2.resize(grad_cam_map, (224, 224))
    
    # Convert input image to numpy for visualization
    input_image_np = np.transpose(input_image.cpu().numpy()[0], (1, 2, 0))
    input_image_np = (input_image_np - input_image_np.min()) / (input_image_np.max() - input_image_np.min())
    
    # Create heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * grad_cam_map), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    overlay = heatmap + input_image_np
    overlay = overlay / np.max(overlay)
    
    # Plot with confidence
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(input_image_np)
    plt.title(f"Original Image\nTrue: {CLASSES[true_label]}")
    plt.axis("off")
    
    plt.subplot(1, 3, 2)
    plt.imshow(grad_cam_map, cmap="jet")
    plt.title("Bias-Aware Grad-CAM Heatmap")
    plt.axis("off")
    
    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    plt.title(f"Overlay\nPredicted: {CLASSES[predicted_class]}\nConfidence: {confidence:.2f}")
    plt.axis("off")
    
    plt.show()

# ============================================================
# 13. Simulated Human-in-the-Loop Feedback
# ============================================================
# For demonstration: Select one example and simulate feedback
feedback_logs = []
example_idx = 0  # Use the first example
input_image = all_images[example_idx].unsqueeze(0).to(device)
true_label = all_labels[example_idx]
grad_cam_map, predicted_class, output = grad_cam.generate(input_image)
confidence = F.softmax(output, dim=1)[0, predicted_class].item()

# Simulate clinician feedback (in practice, replace with input from dashboard)
print(f"Example: True: {CLASSES[true_label]}, Predicted: {CLASSES[predicted_class]}, Confidence: {confidence:.2f}")
feedback = input("Clinician Feedback (e.g., 'correct', 'incorrect - missed tumor', 'flag bias'): ")  # Simulated input

feedback_entry = {
    'example_id': example_idx,
    'true_label': CLASSES[true_label],
    'predicted_label': CLASSES[predicted_class],
    'confidence': confidence,
    'feedback': feedback
}
feedback_logs.append(feedback_entry)

# Save feedback logs
feedback_df = pd.DataFrame(feedback_logs)
feedback_df.to_csv(f"feedback_logs_{timestamp}.csv", index=False)
print(f"Feedback logs saved to feedback_logs_{timestamp}.csv")